In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import requests
import urllib.request
import time

authors = []
dates = []
statements = []
content = []
sources = []
targets = []


In [ ]:
def scrape_website(page_number):
  pgno = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+ pgno +'&ruling=pants-fire'
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text,'html.parser')
  #Get location
  statement_footer = soup.find_all('footer',attrs ={'class':'m-statement__footer'})
  statement_quote = soup.find_all('div',attrs={'class':'m-statement__quote'})
  statement_meta = soup.find_all('div',attrs={'class':'m-statement__meta'})
  target = soup.find_all('div',attrs={'class':'m-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date = link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name+' '+last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month + ' '+ day + ' '+year
    dates.append(date)
    authors.append(full_name)

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)
    atag=i.find_all('a')[0]
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    URL2 = 'https://www.politifact.com' +  atag['href']
    webpage = requests.get(URL2)
    soup2 = BeautifulSoup(webpage.text)
    statement_content = soup2.find_all('article',attrs={'class':'m-textblock'})
    content.append(''.join(statement_content[0].text.strip().splitlines()))


  for i in statement_quote:
   atag=i.find_all('a')[0]   

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
      link4 = i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
      targets.append(link4)



In [ ]:
n = 50
for i in range(1,n):
  scrape_website(i)

In [ ]:
data = pd.DataFrame(columns = ['author','statement','article','source','date','target'])
data['author'] = pd.Series(authors)
data['statement']= pd.Series(statements)
data['source']= pd.Series(sources)
data['article']= pd.Series(content)
data['date']= pd.Series(dates)
data['target']= pd.Series(targets)

data

,author,statement,article,source,date,target
0,Jeff Cercone,“Joe Biden put his pants on backwards”,Attempts on social media to make President Joe...,Instagram posts,"February 10, 2023",pants-fire
1,Ciara O'Rourke,Photos show someone in a President Joe Biden m...,"U.S. Sen. Bernie Sanders, I-Vt., drew attentio...",Facebook posts,"February 10, 2023",pants-fire
2,Tom Kertscher,"Earthquake in Turkey and Syria is ""a systemic ...",The 7.8-magnitude temblor that killed more tha...,Instagram posts,"February 9, 2023",pants-fire
3,Jeff Cercone,"Image shows a ""Super Bowl script was leaked.”",It’s pretty common for National Football Leagu...,Tweets,"February 7, 2023",pants-fire
4,Sara Swann,“‘Satellites’ are on balloons and not in space.”,Following news that a Chinese spy balloon was ...,Instagram posts,"February 7, 2023",pants-fire
...,...,...,...,...,...,...
2965,W. Gardner,NaN,NaN,NaN,"• April 11,",NaN
2966,Amy Sherman,NaN,NaN,NaN,"April 11, 2018",NaN
2967,Mica Soellner,NaN,NaN,NaN,"April 11, 2018",NaN
2968,Amy Sherman,NaN,NaN,NaN,"April 11, 2018",NaN


In [ ]:
data.to_csv('New_False3.csv')